In [1]:
# 02_main.ipynb
from grav_lens import get_datasets

from utils.model import create_model
from utils.loadsave import load_model_with_hyperparameters, load_hyperparameters, save_hyperparameters
from utils.optimize import dimensions, default_parameters

import time
import pickle

from tensorflow.keras import backend as K

from functools import partial

In [7]:
import os 

# mi carpeta data se encuentra en el root
home_data = os.path.join("..", "..")
# Ejemplo de uso
# INDEX 0 es una carpeta de datasets mucho mas chica
train_dataset, val_dataset, test_dataset = get_datasets(data_index='1', max_files=60, 
    home=home_data)

for X, Y in train_dataset.take(1):  # Mostrar un batch de entrenamiento
    print("Train X:", X.shape)
    print("Train Y:", Y.shape)

for X, Y in val_dataset.take(1):  # Mostrar un batch de validación
    print("Val X:", X.shape)
    print("Val Y:", Y.shape)

for X, Y in test_dataset.take(1):  # Mostrar un batch de prueba
    print("Test X:", X.shape)
    print("Test Y:", Y.shape)


Using data folder: ..\..\data\1
Train X: (32, 128, 128, 3)
Train Y: (32, 1, 128, 128, 1)
Val X: (12, 128, 128, 3)
Val Y: (12, 1, 128, 128, 1)
Test X: (6, 128, 128, 3)
Test Y: (6, 1, 128, 128, 1)


In [8]:
# Función de mapeo para las salidas
def map_to_multiple_outputs(X, y):
    return X, {'output_image': y, 'output_mae': y}  # Mapea y_true a todas las salidas

# Aplicar la función de mapeo al dataset
train_dataset = train_dataset.map(map_to_multiple_outputs)

# Arquitectura para lab
Aqui es posible tener acceso a lo que seria el `model.py`, sin necesaidad de resetear todo el tiempo

In [9]:
import tensorflow as tf

@tf.function
def dice_loss_bin(y_true, y_pred):
    mean_true = tf.reduce_mean(y_true)
    mean_pred = tf.reduce_mean(y_pred)
 
    y_true_bin = tf.cast(y_true > mean_true, tf.float32)
    y_pred_bin = tf.cast(y_pred > mean_pred, tf.float32)
 
    intersection = tf.reduce_sum(y_true_bin * y_pred_bin)
    dice_coefficient = (2. * intersection + 1e-7) / (tf.reduce_sum(y_true_bin) + tf.reduce_sum(y_pred_bin) + 1e-7)
 
    return 1 - dice_coefficient


mape_callable = tf.keras.losses.MeanAbsolutePercentageError(
    reduction='sum_over_batch_size',
    name='mean_absolute_percentage_error'
)


def combined_loss(weight_kl=0.1, weight_dice = 32768, weight_mape = 1):
    #  32768 es igual al MAE maximo asumiendo ranogs (-1,1) con 128x128 pixeles
    def loss(y_true, y_pred):
        mae_loss = tf.keras.losses.mae(y_true, y_pred)
        # kl_loss = tf.keras.losses.kld(y_true, y_pred) # requeire modificacion
        dice_loss = dice_loss_bin(y_true, y_pred)
        mape_loss = mape_callable(y_true, y_pred)
        return mae_loss + weight_kl * kl_loss + weight_dice * dice_loss + weight_mape * mape_loss
    return loss

class IdentityLayer(tf.keras.layers.Layer):
    def call(self, inputs):
        return inputs

def create_model(learning_rate=1e-4, 
                 h_kernel_size=3, 
                 hidden_filters=64,
                 out_kernel_size=3,
                 beta_1=0.9,
                 beta_2=0.999,
                 epsilon=1e-7,
                 decay_steps=10000,
                 decay_rate=0.96):
    """
    Modelo que agranda las dimensiones espaciales hasta 256x256 y luego las reduce a 128x128.
    """

    input_layer = tf.keras.layers.Input(shape=(128, 128, 3))

    x = tf.keras.layers.Conv2D(32, (3, 3), activation="sigmoid", padding='same')(input_layer)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.UpSampling2D(size=(2, 2))(x)
    x = tf.keras.layers.Conv2D(hidden_filters, (h_kernel_size, h_kernel_size), activation="sigmoid", padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(2 * hidden_filters, (h_kernel_size, h_kernel_size), activation="sigmoid", padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation="tanh", padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    output_image = tf.keras.layers.Conv2D(1, (out_kernel_size, out_kernel_size), activation="tanh", padding='same', name='output_image')(x)


    # Salidas adicionales para las pérdidas, usando la capa de identidad personalizada
    output_mae = tf.keras.layers.Lambda(lambda x: x, name='output_mae')(output_image)
    #output_mae = IdentityLayer(name='output_mae')(output_image)
    output_mse = IdentityLayer(name='output_mse')(output_image)
    output_logcosh = IdentityLayer(name='output_logcosh')(output_image)
    output_dice = IdentityLayer(name='output_dice')(output_image)

    model = tf.keras.models.Model(inputs=input_layer, 
        outputs=[output_image, output_mae])
        #outputs=[output_image, output_mae, output_mse, output_logcosh, output_dice])



    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
      initial_learning_rate=learning_rate,
      decay_steps=decay_steps,
      decay_rate=decay_rate,
      staircase=True)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, 
                                         beta_1=beta_1, 
                                         beta_2=beta_2, 
                                         epsilon=epsilon)

    # Compile con múltiples funciones de pérdida
    model.compile(optimizer=optimizer, 
                  loss={'output_image': 'mae',
                        'output_mae': 'mae'}, 
                        # 'output_mse': 'mse', 
                        # 'output_logcosh': 'logcosh', 
                        # 'output_dice': dice_loss_bin},
                  loss_weights= {'output_image': 1.0, # Prioridad a la salida principal
                                'output_mae': 1.0})
                                #{'output_image': 1.0}  # Prioridad a la salida principal
                                # 'output_mae': 1.0,  # Las demás se usan para cálculo de pérdidas solamente
                                # 'output_mse': 1.0,
                                # 'output_logcosh': 1.0,
                                # 'output_dice': 1.0})


    return model


# Creando y entrenando un modelo

In [10]:
folder = 'results_2_dani'
hyperparameter = load_hyperparameters(2, folder)
model = create_model(**hyperparameter)

model.fit(train_dataset, epochs=1, verbose=True)

      1/Unknown 28s 28s/step - loss: 1.3072

KeyboardInterrupt: 

In [ ]:



loss = model.evaluate(val_dataset, verbose=False)

OperatorNotAllowedInGraphError: Iterating over a symbolic `tf.Tensor` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating this function with @tf.function. If that does not work, then you may be using an unsupported feature or your source code may not be visible to AutoGraph. See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations.md#access-to-source-code for more information.